In [107]:
import pandas as pd
import urllib
import requests
from pathlib import Path
import numpy as np
import os
import json
from bs4 import BeautifulSoup
import re

# Data Wrangling

1. Unify the file formats (i.e., `json` --> `csv`)
1. Merge metadata from multiple sources (i.e., NIH Common Fund repositories and Database Commons)
1. Add URLs of resources if missing (i.e., Journal homepages using `Sourceid`)
1. Add our own IDs for individual resources by updating (`global_data-portal_id_map.csv`)
1. Add resource connection status

**To-Do**
- [ ] Merge NIH data portals

In [108]:
EVALUATION_DATE_FOLDER = 'Nov-21-2023'

## Data Portals

In [109]:
"""
Load the original data
"""
df = pd.read_json(os.path.join('../input', EVALUATION_DATE_FOLDER, 'database-commons.json'))

# df = df.head(10) # for testing purposes

In [110]:
"""
We use underscore lowercase column names
"""
df.columns = (df.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower())

In [111]:
"""
Drop columns that we don't need
"""
df.drop(columns=['biodb_ranks', 'rating_list'], inplace=True)

In [112]:
"""
The id from the sources are consistently "source_id"
The values should be a string type, and it has the prefix that represents the source (e.g. dc_ for Database Commons)
"""
df.rename(columns={ "db_id": "source_id" }, inplace=True)
df.source_id = df.source_id.apply(lambda x: 'dc_' + str(x))

In [113]:
"""
Some columns from data commons are in json format, we need to convert them to string
Example: [{ "id": 1, "name": "foo" }, { "id": 2, "name": "bar" }] --> 'foo, bar'
"""
json_column_names_and_keys = {
    'data_type_list': 'datatypeName', 
    'category_list': 'name',
    'keywords_list': 'name',
    'data_object_list': 'name',
    'organism_list': 'organismName',
    'theme_list': 'name'
}

for (column, key) in json_column_names_and_keys.items():
    df[column] = df[column].apply(lambda x: ', '.join([object[key] for object in x]))

In [114]:
# This takes too much time. We will instead just run WAVE API requests for all URLs.
# # Check the connection status and put that as a `reachable` column

# # TODO: a faster way to do this?
# def check_connection_status(url):
#     print(url)
#     try:
#         status = requests.get(url)
#     except Exception:
#         return False
#     return status.status_code == 200
    
# df['reachable'] = df['url'].apply(lambda x: check_connection_status(x))

### Update the ID Mapping Table (`data-portal_id_map.csv`)

In [115]:
"""
create or update a mapping table (i.e., `data-portal_id_map.csv`)
"""
# Load the data first
file = Path(os.path.join('../output', 'data-portal_id_map.csv'))
if file.exists():
    df_map = pd.read_csv(file)
else:
    df_map = pd.DataFrame(columns=['id', 'source_id', 'date_added'])

# Find rows that does not already exist in the table
df_temp = df_map.merge(df, how='outer', on='source_id', indicator=True)
df_new_resources = df_temp[df_temp['_merge'] == 'right_only'][['source_id']]

# Ensure to assign new `id`s, i.e., New `id` == max id + 1
max_id = df_map.id.max()
max_id = 0 if max_id is np.nan else max_id
new_id = max_id + 1

df_new_resources.insert(0, 'id', range(new_id, new_id + len(df_new_resources)))
df_new_resources['date_added'] = pd.to_datetime('today').strftime('%m-%d-%Y')

pd.concat([df_map, df_new_resources], axis=0).to_csv(file, index=False)

### Save the Metadata of Data Portals (`data-portal_metadata.csv`)

In [117]:
"""
Add the `id` column to the original metadata
"""
df_map = pd.read_csv(file)
df_meta = df_map[['id', 'source_id']].merge(df, how='right', on='source_id')
df_meta.to_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'data-portal_metadata.csv'), index=False)
df_meta.head(1)

,id,source_id,short_name,accession,full_name,description,url,founded_year,host_institution,address,...,category_list,keywords_list,data_object_list,organism_list,theme_list,zindex,first_publication_year,search_example,cited_date,ess
0,1,dc_3061,DAVID,DBC003061,"Database for Annotation, Visualization and Int...","The Database for Annotation, Visualization and...",https://david.ncifcrf.gov,2003,National Cancer Institute,Laboratory of Immunopathogenesis and Bioinfor...,...,"Pathway, Standard ontology and nomenclature",functional annotation,"Animal, Plant, Fungi, Bacteria, Archaea","Homo sapiens, Mus musculus",,1847.8,2007,None,2022-12-04 00:01:00,0


### Save the Pages of Data Portals (`data-portal_pages.csv`)

In [118]:
"""
Create `data-portal_pages.csv` and add homepage urls.
"""
df_meta = pd.read_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'data-portal_metadata.csv'))

df_pages = df_meta[['id', 'url']].copy()

df_pages['page_type'] = 'home'
df_pages['page_id'] = df_pages['id']
df_pages['page_id'] = df_pages['page_id'].apply(lambda x: str(x) + '_' + 'home')

df_pages.to_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'data-portal_pages.csv'), index=False)

## Journals

In [120]:
"""
Load the original data
"""
df = pd.read_csv(os.path.join('../input', EVALUATION_DATE_FOLDER, 'scimagojr 2022.csv'), sep=';')

# df = df.head(5) # for testing purposes

In [121]:
"""
Using `Sourceid` of SJR, get URLs of individual journal portals
TODO: Reuse the previously identified home pages
"""
def infer_homepage(Sourceid):
    info_url = f'https://www.scimagojr.com/journalsearch.php?q={Sourceid}&tip=sid&clean=0'
    html_text = requests.get(info_url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    urls = soup.find_all('a', text=re.compile('Homepage'))
    if len(urls) > 0:
        return urls[0].get('href')
    else:
        print(f'No homepage found for {Sourceid}')
        return None

df['url'] = df['Sourceid'].apply(lambda x: infer_homepage(x))

/var/folders/rs/7nt0b63d5pn3m6rd7m8zt_6r0000gn/T/ipykernel_16924/1668964650.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  urls = soup.find_all('a', text=re.compile('Homepage'))


No homepage found for 30611
No homepage found for 110362
No homepage found for 21100239245
No homepage found for 19700175113
No homepage found for 81474
No homepage found for 5000158305
No homepage found for 71628
No homepage found for 25392
No homepage found for 21100784787
No homepage found for 21101070303
No homepage found for 21101019253
No homepage found for 21100983356
No homepage found for 27392
No homepage found for 21101047803
No homepage found for 21100894516
No homepage found for 19900191708
No homepage found for 21100851290
No homepage found for 21100223326
No homepage found for 21100851285
No homepage found for 18264
No homepage found for 21100896491
No homepage found for 21100239235
No homepage found for 21100905330
No homepage found for 14395
No homepage found for 21100784717
No homepage found for 22545
No homepage found for 21100243806
No homepage found for 83986
No homepage found for 21101058912
No homepage found for 21101081510
No homepage found for 21100314711
No hom

In [127]:
"""
We use underscore lowercase column names
"""
df.columns = (df.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower())
df.columns = (df.columns.str.replace('.', '')) # remove dots
df.columns = (df.columns.str.replace('(', '')) # remove parentheses
df.columns = (df.columns.str.replace(')', ''))
df.columns = (df.columns.str.replace('/', 'per')) # replace slash with "per"
df.columns = (df.columns.str.replace(' ', '_')) # replace space with underscore

In [128]:
"""
The id from the sources are consistently "source_id"
The values should be a string type, and it has the prefix that represents the source (e.g. dc_ for Database Commons)
"""
df.rename(columns={ "sourceid": "source_id" }, inplace=True)
df.source_id = df.source_id.apply(lambda x: 'sjr_' + str(x))

### Update the ID Mapping Table (`journal-portal_id_map.csv`)

In [129]:
"""
create or update a mapping table (i.e., `data-portal_id_map.csv`)
"""
# Load the data first
file = Path(os.path.join('../output', 'journal-portal_id_map.csv'))
if file.exists():
    df_map = pd.read_csv(file)
else:
    df_map = pd.DataFrame(columns=['id', 'source_id', 'date_added'])

# Find rows that does not already exist in the table
df_temp = df_map.merge(df, how='outer', on='source_id', indicator=True)
df_new_resources = df_temp[df_temp['_merge'] == 'right_only'][['source_id']]

# Ensure to assign new `id`s, i.e., New `id` == max id + 1
max_id = df_map.id.max()
max_id = 0 if max_id is np.nan else max_id
new_id = max_id + 1

df_new_resources.insert(0, 'id', range(new_id, new_id + len(df_new_resources)))
df_new_resources['date_added'] = pd.to_datetime('today').strftime('%m-%d-%Y')

pd.concat([df_map, df_new_resources], axis=0).to_csv(file, index=False)

### Save the Metadata of Journal Portals (`journal-portal_metadata.csv`)

In [130]:
"""
Add the `id` column to the original metadata
"""
df_map = pd.read_csv(file)
df_meta = df_map[['id', 'source_id']].merge(df, how='right', on='source_id')
df_meta.to_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'journal-portal_metadata.csv'), index=False)
df_meta.head(10)

,id,source_id,rank,title,type,issn,sjr,sjr_best_quartile,h_index,total_docs_2022,...,citable_docs_3years,cites_per_doc_2years,ref_per_doc,country,region,publisher,coverage,categories,areas,url
0,27956,sjr_sjr_28773,1,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","86,091",Q1,198,44,...,85,"299,99","97,00",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine,https://onlinelibrary.wiley.com/journal/15424863
1,27957,sjr_sjr_29431,2,Quarterly Journal of Economics,journal,"00335533, 15314650","36,730",Q1,292,36,...,122,"14,83","66,61",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance",https://academic.oup.com/qje
2,27958,sjr_sjr_20315,3,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","34,201",Q1,485,121,...,156,"35,47","89,29",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology",https://www.nature.com/nrm/
3,27959,sjr_sjr_18434,4,Cell,journal,"00928674, 10974172","26,494",Q1,856,420,...,1440,"43,80","65,74",United States,Northern America,Cell Press,1974-2022,"Biochemistry, Genetics and Molecular Biology (...","Biochemistry, Genetics and Molecular Biology",https://www.cell.com/cell/home
4,27960,sjr_sjr_15847,5,New England Journal of Medicine,journal,"00284793, 15334406","26,015",Q1,1130,1410,...,1854,"33,93","10,21",United States,Northern America,Massachussetts Medical Society,1945-2022,Medicine (miscellaneous) (Q1),Medicine,https://www.nejm.org/
5,27961,sjr_sjr_15819,6,Nature Medicine,journal,"1546170X, 10788956","24,687",Q1,605,522,...,664,"34,67","29,23",United Kingdom,Western Europe,Nature Publishing Group,1995-2022,"Biochemistry, Genetics and Molecular Biology (...","Biochemistry, Genetics and Molecular Biology; ...",https://www.nature.com/nm/
6,27962,sjr_sjr_19434,7,MMWR Recommendations and Reports,journal,"10575987, 15458601","23,962",Q1,151,103,...,133,"25,28","9,13",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2022,Epidemiology (Q1); Health Information Manageme...,Environmental Science; Health Professions; Med...,https://www.cdc.gov/mmwr/indrr_2020.html
7,27963,sjr_sjr_16115,8,Nature Biotechnology,journal,"15461696, 10870156","22,781",Q1,491,423,...,426,"16,35","25,65",United Kingdom,Western Europe,Nature Publishing Group,1989-2022,Applied Microbiology and Biotechnology (Q1); B...,"Biochemistry, Genetics and Molecular Biology; ...",https://www.nature.com/nbt/
8,27964,sjr_sjr_21100812243,9,Nature Reviews Materials,journal,20588437,"21,927",Q1,156,122,...,168,"40,82","90,43",United Kingdom,Western Europe,Nature Publishing Group,2016-2022,"Biomaterials (Q1); Electronic, Optical and Mag...",Energy; Materials Science,https://www.nature.com/natrevmats/
9,27965,sjr_sjr_22697,10,American Economic Review,journal,00028282,"21,833",Q1,337,71,...,354,"9,78","59,68",United States,Northern America,American Economic Association,"1973-1975, 1978-2022",Economics and Econometrics (Q1),"Economics, Econometrics and Finance",https://www.aeaweb.org/journals/aer


### Save the Pages of Journal Portals (`journal-portal_pages.csv`)

In [131]:
"""
Create `*_pages.csv` and add homepage urls.
"""
df_meta = pd.read_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'journal-portal_metadata.csv'))

df_pages = df_meta[['id', 'url']].copy()

df_pages['page_type'] = 'home'
df_pages['page_id'] = df_pages['id']
df_pages['page_id'] = df_pages['page_id'].apply(lambda x: str(x) + '_' + 'home')

df_pages.to_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'journal-portal_pages.csv'), index=False)